In [443]:
import pandas as pd
from collections import defaultdict

In [488]:
exit_price = 175
fin_independence = 15
tax = .5

equity_holders_definitions = ['Equity Percent [0]', 'Subject to Progressive [1]', 'Triggered [2]',
                             'Employed [3]', 'Payout [4]']

equity_holders = {'angel1' :  [.1, False, False, False, 0 ],
                 'venture1':  [.1, False, False, False, 0],
                 'founder1':  [.25, True, False, True, 0],
                 'founder2':  [.25, True, False, True, 0],
                 'employee1': [.1, True, False, True, 0],
                 'employee2': [.1, True, False, False, 0],
                 'employee3': [.05, True, False, True, 0],
                 'employee4': [.05, True, False, False, 0]}


In [489]:
def check_equity_balance(equity_holders):
    '''
    Checks to ensure that the equity values inputted add up to 1. 
    
    '''
    total_equity = 0
    for data in equity_holders.values():
        equity = data[0]
        total_equity += equity    
        
    if total_equity != 1:
        print("Error in equity input. Total equity does not add up to 1. \n" 
              "Total equity = ", total_equity, '.')
        return False
    else:
        return True
        

In [490]:
print(check_equity_balance(equity_holders))

True


In [491]:
def round_1(equity_holders, exit_price, fin_independence, tax ):
    pool = 0
    
    for name, data in equity_holders.items():
        equity_percent = data[0]
        subject_progressive = data[1]
        payout = (equity_percent * exit_price)
        if subject_progressive == False:
            equity_holders[name][4] = payout
        
        else:
            if payout > fin_independence:
                tax = ((payout - fin_independence) / 2)
                pool += tax
                payout -= tax
                equity_holders[name][4] = payout                
                equity_holders[name][2] = True
            else:
                equity_holders[name][4] = payout
    return pool, equity_holders

In [492]:
def redistribution_denominator(pool, equity_holders, fin_independence):
    equity_denominator = 0
    for name, data in equity_holders.items():
        equity = data[0]
        triggered = data[2]
        employed = data[3]
        if triggered == False and employed == True:
            equity_denominator += equity
                
    return equity_denominator
    


In [493]:
    def distribute_pool(pool, equity_holders, fin_independence, equity_denominator):
        '''

        '''
        distributed_funds = 0
        for name, data in equity_holders.items():
            equity = data[0]
            triggered = data[2]
            employed = data[3]
            payout = data[4]
            if triggered == False and employed == True:
                max_redistribution = (equity / equity_denominator * pool + payout)
                fin_independence_gap = fin_independence - payout
                redistribution = min(fin_independence_gap, max_redistribution)
                distributed_funds += redistribution
                equity_holders[name][4] = payout + redistribution
                equity_holders[name][2] = True

        pool -= distributed_funds        

        return pool, equity_holders

In [494]:
def final_distribution(equity_holders, pool):
    equity_denominator = 0
    for name, data in equity_holders.items():
        equity = data[0]
        progressive = data[1]
        employed = data[3]
        if progressive == True and employed == True:
            equity_denominator += equity
        
    for name, data in equity_holders.items():
        equity = data[0]
        progressive = data[1]
        employed = data[3]
        payout = data[4]
        if progressive == True and employed == True:
            distribution = equity / equity_denominator * pool
            equity_holders[name][4] += distribution
        
    return equity_holders 

In [495]:
def redistribution (equity_holders, pool, fin_independence):
    equity_denominator = redistribution_denominator(pool, equity_holders, fin_independence)
    if equity_denominator > 0:
        pool, equity_holders = distribute_pool(pool, equity_holders, fin_independence, equity_denominator)
        print("Pool: ", pool)
        print(equity_holders)
        if pool > 0:
            redistribution(equity_holders, pool, fin_independence)
            return equity_holders
        else:
            return equity_holders
    
    else:
        equity_holders = final_distribution(equity_holders, pool)
        return pool, equity_holders



In [496]:
def progressive_payout(exit_price, fin_independence, equity_holders, tax):  
    check_equity_balance(equity_holders)
    
    pool, equity_holders = round_1(equity_holders, exit_price, fin_independence, tax)
    print("Pool: ", pool)
    print(equity_holders)
        
    if pool > 0:
        equity_holders = redistribution(equity_holders, pool, fin_independence)
        return pool, equity_holders
            
    else:
        return equity_holders
        
    

In [497]:
progressive_payout(exit_price, fin_independence, equity_holders, tax)

Pool:  31.25
{'angel1': [0.1, False, False, False, 17.5], 'venture1': [0.1, False, False, False, 17.5], 'founder1': [0.25, True, True, True, 29.375], 'founder2': [0.25, True, True, True, 29.375], 'employee1': [0.1, True, True, True, 16.25], 'employee2': [0.1, True, True, False, 16.25], 'employee3': [0.05, True, False, True, 8.75], 'employee4': [0.05, True, False, False, 8.75]}
Pool:  25.0
{'angel1': [0.1, False, False, False, 17.5], 'venture1': [0.1, False, False, False, 17.5], 'founder1': [0.25, True, True, True, 29.375], 'founder2': [0.25, True, True, True, 29.375], 'employee1': [0.1, True, True, True, 16.25], 'employee2': [0.1, True, True, False, 16.25], 'employee3': [0.05, True, True, True, 15.0], 'employee4': [0.05, True, False, False, 8.75]}


(31.25,
 {'angel1': [0.1, False, False, False, 17.5],
  'employee1': [0.1, True, True, True, 20.096153846153847],
  'employee2': [0.1, True, True, False, 16.25],
  'employee3': [0.05, True, True, True, 16.923076923076923],
  'employee4': [0.05, True, False, False, 8.75],
  'founder1': [0.25, True, True, True, 38.99038461538461],
  'founder2': [0.25, True, True, True, 38.99038461538461],
  'venture1': [0.1, False, False, False, 17.5]})